In [ ]:
import os

# Set proxy configuration for network requests
os.environ["http_proxy"] = "http://proxy.ims.intel.com:911"
os.environ["https_proxy"] = "http://proxy.ims.intel.com:911"

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

# Initialize HuggingFace embeddings model
embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-large")

# Load text file content
loader = TextLoader("results.txt")
docs = loader.load()

# Configure text splitter to split by lines
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1,  # Each chunk is 1 line
    chunk_overlap=0,
    separators=["\n"],  # Split on newlines only
    add_start_index=True,
)

splits = text_splitter.split_documents(docs)


# Create FAISS vector store and retriever
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [6]:
from langchain_core.prompts import PromptTemplate

# Define prompt for question-answering
custom_prompt_template = """
### You are a helpful AI assistant that identifies whether a failed unit test in a pull request CI test matches any known github issues in the knowledge base. Follow this structured approach:\n\n \
1. **Extract Test Information**\n   \
- Identify the operation(s) under test (e.g., 'add' from test_compare_cpu_add_xpu)\n   \
- Extract key details from the error message and test case (test class, dtype, components, scenario)\n\n   \
2. **Knowledge Base Analysis**\n   \
- Compare against search results using these criteria:\n  \
* Operation match (must be same ops)\n     \
* Error message similarity\n     \
* Test case characteristics\n     \
* Randomness consistency (both random or both deterministic)\n\n   \
3. **Similarity Determination**\n   \
- For each candidate issue from knowledge base:\n     \
* Mark as similar ONLY if:\n       \
- Same operation under test\n       \
- Matching error patterns\n       \
- Comparable test scenario\n      \
- Consistent randomness\n     \
* Explicitly exclude matches with:\n    \
- The pull request of the input question\n \
- Mismatched operations\n       \
- Mismatched randomness\n    \
- Incompatible test scenarios\n\n       \
4. **Output Structured Response**\n   \
- If similar issue found:\n     \
* Provide issue ID, state, and ops\n     \
* Inherit relevant metadata\n   \
- If no match:\n     \
* Set similar_issue_id and state to 'N/A'\n    \
* Propose new issue details\n\n    \
5. **Response Format Requirements**\n     \
- Strictly use this JSON structure with fileds:\n   \
-- similar_issue_id\n     \
-- similar_issue_state\n     \
-- ops_to_test\n     \
-- issue_owner\n     \
-- issue_description\n     \
-- root_causes\n     \
-- suggested_solutions\n   \
- Only include verified information\n   \
- Never hallucinate missing details\n\n    \
6. **Special Considerations**\n   \
- Cross-check test class hierarchy\n   \
- Verify dtype compatibility\n   \
- Note component dependencies\n   \
- Flag partial matches as non-similar\n\n
### Search results: {context} 
### Question: {question}
### Answer:
"""

prompt = PromptTemplate.from_template(custom_prompt_template)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key="EMPTY",
    openai_api_base="http://skyrex.jf.intel.com:8000/v1",
    model_name="meta-llama/Llama-3.3-70B-Instruct",
    max_tokens=2000,
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


answer = rag_chain.invoke(
    "test test_exp_xpu_float32 failed, what is the reason of the failure?"
)
print(answer)